#### Approach 
- Reading corpus
- Tokenization
- Lemmatization
- Options for generatinng response
    - Bag of words
    - Tf-Idf
    - Cosine similarity
    - Dense layer model
- Open chat in jupyter notebook
- Create UI for chat bot

In [15]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
from nltk.chat.util import Chat, reflections
import copy

[nltk_data] Downloading package punkt to C:\Users\Manoj-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Manoj-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import json

In [10]:
f=open('GL Bot.json',)
data=json.load(f)
f.close()

print(type(data))

<class 'dict'>


In [11]:
data.keys()

dict_keys(['intents'])

In [12]:
for i in data['intents']:
    print(i)
    print()

{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?'], 'context_set': ''}

{'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye'], 'context_set': ''}

{'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for olympus', 'lot of problem with o

In [16]:

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
intents = copy.deepcopy(data)
#keeping org data 

In [17]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # tokenize word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        documents.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [18]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

128 documents
8 classes ['Bot', 'Exit', 'Intro', 'NN', 'Olympus', 'Profane', 'SL', 'Ticket']
158 unique lemmatized words ['a', 'able', 'access', 'activation', 'ada', 'adam', 'aifl', 'aiml', 'am', 'an', 'ann', 'anyone', 'are', 'artificial', 'backward', 'bad', 'bagging', 'batch', 'bayes', 'belong', 'best', 'blended', 'bloody', 'boosting', 'bot', 'buddy', 'classification', 'contact', 'create', 'cross', 'cya', 'day', 'deep', 'did', 'diffult', 'do', 'ensemble', 'epoch', 'explain', 'first', 'for', 'forest', 'forward', 'from', 'function', 'good', 'goodbye', 'gradient', 'great', 'hate', 'have', 'hell', 'hello', 'help', 'helped', 'hey', 'hi', 'hidden', 'hour', 'how', 'hyper', 'i', 'imputer', 'in', 'intelligence', 'is', 'jerk', 'joke', 'knn', 'later', 'layer', 'learner', 'learning', 'leaving', 'link', 'listen', 'logistic', 'lot', 'machine', 'me', 'ml', 'my', 'naive', 'name', 'nb', 'net', 'network', 'neural', 'no', 'not', 'of', 'olympus', 'olypus', 'on', 'online', 'operation', 'opertions', 'otimi

In [19]:

training = []
output_empty = [0] * len(classes)
for doc in documents:
    
    bag = []
    
    pattern_words = doc[0]
    # lemmatize
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [20]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
128/128 [==============================] - 0s 3ms/step - loss: 2.0778 - accuracy: 0.1719
Epoch 2/200
128/128 [==============================] - 0s 437us/step - loss: 1.9282 - accuracy: 0.2891
Epoch 3/200
128/128 [==============================] - 0s 437us/step - loss: 1.8977 - accuracy: 0.3047
Epoch 4/200
128/128 [==============================] - 0s 500us/step - loss: 1.7859 - accuracy: 0.3125
Epoch 5/200
128/128 [==============================] - 0s 375us/step - loss: 1.7201 - accuracy: 0.3828
Epoch 6/200
128/128 [==============================] - 0s 437us/step - loss: 1.6165 - accuracy: 0.4688
Epoch 7/200
128/128 [==============================] - 0s 375us/step - loss: 1.5060 - accuracy: 0.5078
Epoch 8/200
128/128 [==============================] - 0s 437us/step - loss: 1.3373 - accuracy: 0.4922
Epoch 9/200
128/128 [==============================] - 0s 375us/step - loss: 1.2432 - accuracy: 0.5547
Epoch 10/200
128/128 [==============================] - 0s 375us/step - los

128/128 [==============================] - 0s 437us/step - loss: 0.0121 - accuracy: 1.0000
Epoch 158/200
128/128 [==============================] - 0s 437us/step - loss: 0.0103 - accuracy: 1.0000
Epoch 159/200
128/128 [==============================] - 0s 375us/step - loss: 0.0248 - accuracy: 0.9922
Epoch 160/200
128/128 [==============================] - 0s 375us/step - loss: 0.0043 - accuracy: 1.0000
Epoch 161/200
128/128 [==============================] - 0s 375us/step - loss: 0.0117 - accuracy: 1.0000
Epoch 162/200
128/128 [==============================] - 0s 375us/step - loss: 0.0095 - accuracy: 1.0000
Epoch 163/200
128/128 [==============================] - 0s 375us/step - loss: 0.0266 - accuracy: 0.9844
Epoch 164/200
128/128 [==============================] - 0s 375us/step - loss: 0.0169 - accuracy: 0.9922
Epoch 165/200
128/128 [==============================] - 0s 437us/step - loss: 0.0089 - accuracy: 1.0000
Epoch 166/200
128/128 [==============================] - 0s 375us/ste

In [23]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('GL Bot.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [24]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


In [25]:

def bow(sentence, words, show_details=True):
    
    sentence_words = clean_up_sentence(sentence)
    
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))


In [26]:
def predict_class(sentence, model):
    
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [27]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [29]:
flag=True
print("AutoBot: My name is AutoBot. I will try to assist with your queries about GL. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("AutoBot: You are welcome..")
        else:
            if(chatbot_response(user_response)!=None):
                print("AutoBot: "+chatbot_response(user_response))
            else:
                print("AutoBot: ",end="")
                print(chatbot_response(user_response))
                #sent_tokens.remove(user_response)
    else:
        flag=False
        print("AutoBot: Happy Learning. Reach us whenever you face any doubts.")


AutoBot: My name is AutoBot. I will try to assist with your queries about GL. If you want to exit, type Bye!
hello
AutoBot: Hello! how can i help you ?
tree
AutoBot: Link: Machine Learning wiki 
neural network
AutoBot: Link: Neural Nets wiki
exit
AutoBot: Link: Machine Learning wiki 
i dont need links
AutoBot: Hello! how can i help you ?
what you know
AutoBot: Please use respectful words
who are you
AutoBot: I am your virtual learning assistant
Jon snows knows nothing
AutoBot: Link: Machine Learning wiki 
thts quite a response
AutoBot: I hope I was able to assist you, Good Bye
not yet
AutoBot: Tarnsferring the request to your PM
why so serious
AutoBot: Link: Machine Learning wiki 
okay
AutoBot: Link: Machine Learning wiki 
bye
AutoBot: Happy Learning. Reach us whenever you face any doubts.


In [28]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("AutoBot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)


ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)


scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set


SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )


EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)



scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

- References
- https://medium.com/analytics-vidhya/building-a-simple-chatbot-in-python-using-nltk-7c8c8215ac6e